## Classification
Put simply, classification is the task of predicting a label for a given observation. For example: you are given certain physical descriptions of an animal, and your taks is to classify them as either a dog or a cat. Here, we will classify iris flowers.

As we will see later, we will use different classifiers and at the end of this notebook, we will compare them. We will define our accuracy function right now to get it out of the way. We will use a simple accuracy function that returns the ratio of the number of correctly classified observations to the total number of predictions.

In [2]:
findaccuracy(predictedvals,groundtruthvals) = sum(predictedvals.==groundtruthvals)/length(groundtruthvals)

findaccuracy (generic function with 1 method)

In [1]:
using GLMNet
using RDatasets
using MLBase
using Plots
using DecisionTree
using Distances
using NearestNeighbors
using Random
using LinearAlgebra
using DataStructures
using LIBSVM

Get the data first

In [5]:
iris = dataset("datasets", "iris")
describe(iris)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,SepalLength,5.84333,4.3,5.8,7.9,0,Float64
2,SepalWidth,3.05733,2.0,3.0,4.4,0,Float64
3,PetalLength,3.758,1.0,4.35,6.9,0,Float64
4,PetalWidth,1.19933,0.1,1.3,2.5,0,Float64
5,Species,,setosa,,virginica,0,"CategoricalValue{String, UInt8}"


In [4]:
X = Matrix(iris[:,1:4])
irislabels = iris[:,5]

150-element CategoricalArrays.CategoricalArray{String,1,UInt8}:
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 ⋮
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"

In [6]:
X

150×4 Matrix{Float64}:
 5.1  3.5  1.4  0.2
 4.9  3.0  1.4  0.2
 4.7  3.2  1.3  0.2
 4.6  3.1  1.5  0.2
 5.0  3.6  1.4  0.2
 5.4  3.9  1.7  0.4
 4.6  3.4  1.4  0.3
 5.0  3.4  1.5  0.2
 4.4  2.9  1.4  0.2
 4.9  3.1  1.5  0.1
 5.4  3.7  1.5  0.2
 4.8  3.4  1.6  0.2
 4.8  3.0  1.4  0.1
 ⋮              
 6.0  3.0  4.8  1.8
 6.9  3.1  5.4  2.1
 6.7  3.1  5.6  2.4
 6.9  3.1  5.1  2.3
 5.8  2.7  5.1  1.9
 6.8  3.2  5.9  2.3
 6.7  3.3  5.7  2.5
 6.7  3.0  5.2  2.3
 6.3  2.5  5.0  1.9
 6.5  3.0  5.2  2.0
 6.2  3.4  5.4  2.3
 5.9  3.0  5.1  1.8

In [13]:
irislabelsmap = labelmap(irislabels)
y = labelencode(irislabelsmap, irislabels)


150-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3

In classification, we often want to use some of the data to fit a model, and the rest of the data to validate (commonly known as `training` and `testing` data). We will get this data ready now so that we can easily use it in the rest of this notebook.

In [18]:
function perclass_splits(y,at)
    uids = unique(y)
    keepids = []
    for ui in uids
        curids = findall(y.==ui)
        rowids = randsubseq(curids, at) 
        push!(keepids,rowids...)
    end
    return keepids
end

perclass_splits (generic function with 1 method)

In [8]:
?randsubseq

search: randsubseq randsubseq! RandomSub StratifiedRandomSub



```
randsubseq([rng=GLOBAL_RNG,] A, p) -> Vector
```

Return a vector consisting of a random subsequence of the given array `A`, where each element of `A` is included (in order) with independent probability `p`. (Complexity is linear in `p*length(A)`, so this function is efficient even if `p` is small and `A` is large.) Technically, this process is known as "Bernoulli sampling" of `A`.

# Examples

```jldoctest
julia> rng = MersenneTwister(1234);

julia> randsubseq(rng, 1:8, 0.3)
2-element Vector{Int64}:
 7
 8
```


In [20]:
trainids = perclass_splits(y,0.7)
testids = setdiff(1:length(y),trainids)

49-element Vector{Int64}:
   2
   7
   9
  12
  16
  17
  21
  22
  24
  26
  27
  28
  29
   ⋮
 122
 123
 124
 125
 128
 129
 133
 136
 139
 140
 145
 146

In [21]:
?setdiff

search: setdiff setdiff! selectdim setrounding SortedMultiDict searchsortedfirst



```
setdiff(s, itrs...)
```

Construct the set of elements in `s` but not in any of the iterables in `itrs`. Maintain order with arrays.

See also [`setdiff!`](@ref), [`union`](@ref) and [`intersect`](@ref).

# Examples

```jldoctest
julia> setdiff([1,2,3], [3,4,5])
2-element Vector{Int64}:
 1
 2
```

---

```
setdiff(ss1, ss2)
```

The two arguments are sorted sets with the same key and order type. This operation computes the difference, i.e., a sorted set containing entries that in are in `ss1` but not `ss2`. Time: O(*cn* log *n*), where *n* is the total size of the two containers.


We will need one more function, and that is the function that will assign classes based on the predicted values when the predicted values are continuous.

In [48]:
assign_class(predictedvalue) = argmin(abs.(predictedvalue .- [1,2,3]))

assign_class (generic function with 1 method)

### 🟣 Method 1: Lasso

In [22]:
path = glmnet(X[trainids,:], y[trainids])
cv = glmnetcv(X[trainids,:], y[trainids])

Least Squares GLMNet Cross Validation
57 models for 4 predictors in 10 folds
Best λ 0.009 (mean loss 0.054, std 0.008)

In [55]:
# choose the best lambda to predict with.
path = glmnet(X[trainids,:], y[trainids])
cv = glmnetcv(X[trainids,:], y[trainids])
mylambda = path.lambda[argmin(cv.meanloss)]

@show path = glmnet(X[trainids,:], y[trainids],lambda=[mylambda]);

path = glmnet(X[trainids, :], y[trainids], lambda = [mylambda]) = Least Squares GLMNet Solution Path (1 solutions for 4 predictors in 75 passes):
─────────────────────────────
     df   pct_dev           λ
─────────────────────────────
[1]   3  0.919243  0.00450488
─────────────────────────────


In [56]:
q = X[testids,:];
predictions_lasso = GLMNet.predict(path,q)

49×1 Matrix{Float64}:
 0.9798535313064545
 1.0150508757717105
 0.9861597152875221
 0.988970802746817
 1.0295821200328776
 1.0267710325735826
 1.0061418064291332
 1.0737254079003504
 1.1937142315787794
 1.0141955386710872
 1.1098149635258694
 0.9654936150834331
 0.9546287953821844
 ⋮
 2.6810484751611314
 2.990126541442825
 2.566510498363147
 2.84730766510385
 2.547591946419944
 2.8616675813268717
 2.922089661716398
 3.0557543925553703
 2.530420942737628
 2.8084070220190362
 3.088995986661955
 2.9012153594145236

In [57]:
predictions_lasso = assign_class.(predictions_lasso)
findaccuracy(predictions_lasso,y[testids])

0.9591836734693877

### 🟣 Method 2: Ridge
We will use the same function but set alpha to zero.

In [50]:
# choose the best lambda to predict with.
path = glmnet(X[trainids,:], y[trainids],alpha=0);
cv = glmnetcv(X[trainids,:], y[trainids],alpha=0)
mylambda = path.lambda[argmin(cv.meanloss)]
path = glmnet(X[trainids,:], y[trainids],alpha=0,lambda=[mylambda]);
q = X[testids,:];
predictions_ridge = GLMNet.predict(path,q)
predictions_ridge = assign_class.(predictions_ridge)
findaccuracy(predictions_ridge,y[testids])

0.9183673469387755

### 🟣 Method 3: Elastic Net
We will use the same function but set alpha to 0.5 (it's the combination of lasso and ridge).

In [54]:
# choose the best lambda to predict with.
path = glmnet(X[trainids,:], y[trainids],alpha=0.5);
cv = glmnetcv(X[trainids,:], y[trainids],alpha=0.5)
mylambda = path.lambda[argmin(cv.meanloss)]
path = glmnet(X[trainids,:], y[trainids],alpha=0.5,lambda=[mylambda]);
q = X[testids,:];
predictions_EN = GLMNet.predict(path,q)
predictions_EN = assign_class.(predictions_EN)
findaccuracy(predictions_EN,y[testids])

0.9591836734693877

### 🟣 Method 4: Decision Trees
We will use the package `DecisionTree`

In [58]:
model = DecisionTreeClassifier(max_depth=2)
DecisionTree.fit!(model, X[trainids,:], y[trainids])

DecisionTreeClassifier
max_depth:                2
min_samples_leaf:         1
min_samples_split:        2
min_purity_increase:      0.0
pruning_purity_threshold: 1.0
n_subfeatures:            0
classes:                  [1, 2, 3]
root:                     Decision Tree
Leaves: 3
Depth:  2

In [59]:
q = X[testids,:];
predictions_DT = DecisionTree.predict(model, q)
findaccuracy(predictions_DT,y[testids])

0.8979591836734694

### 🟣 Method 5: Random Forests
The `RandomForestClassifier` is available through the `DecisionTree` package as well.

In [60]:
model = RandomForestClassifier(n_trees=20)
DecisionTree.fit!(model, X[trainids,:], y[trainids])

RandomForestClassifier
n_trees:             20
n_subfeatures:       -1
partial_sampling:    0.7
max_depth:           -1
min_samples_leaf:    1
min_samples_split:   2
min_purity_increase: 0.0
classes:             [1, 2, 3]
ensemble:            Ensemble of Decision Trees
Trees:      20
Avg Leaves: 5.55
Avg Depth:  4.2

In [61]:
q = X[testids,:];
predictions_RF = DecisionTree.predict(model, q)
findaccuracy(predictions_RF,y[testids])

0.9183673469387755

### 🟣 Method 6: Using a Nearest Neighbor method
We will use the `NearestNeighbors` package here.

In [62]:
Xtrain = X[trainids,:]
ytrain = y[trainids]
kdtree = KDTree(Xtrain')

KDTree{StaticArrays.SVector{4, Float64}, Euclidean, Float64}
  Number of points: 101
  Dimensions: 4
  Metric: Euclidean(0.0)
  Reordered: true

In [63]:
queries = X[testids,:]

49×4 Matrix{Float64}:
 4.9  3.0  1.4  0.2
 4.6  3.4  1.4  0.3
 4.4  2.9  1.4  0.2
 4.8  3.4  1.6  0.2
 5.7  4.4  1.5  0.4
 5.4  3.9  1.3  0.4
 5.4  3.4  1.7  0.2
 5.1  3.7  1.5  0.4
 5.1  3.3  1.7  0.5
 5.0  3.0  1.6  0.2
 5.0  3.4  1.6  0.4
 5.2  3.5  1.5  0.2
 5.2  3.4  1.4  0.2
 ⋮              
 5.6  2.8  4.9  2.0
 7.7  2.8  6.7  2.0
 6.3  2.7  4.9  1.8
 6.7  3.3  5.7  2.1
 6.1  3.0  4.9  1.8
 6.4  2.8  5.6  2.1
 6.4  2.8  5.6  2.2
 7.7  3.0  6.1  2.3
 6.0  3.0  4.8  1.8
 6.9  3.1  5.4  2.1
 6.7  3.3  5.7  2.5
 6.7  3.0  5.2  2.3

In [64]:
idxs, dists = knn(kdtree, queries', 5, true)

([[21, 29, 9, 7, 18], [30, 2, 27, 17, 3], [24, 3, 27, 10, 30], [17, 6, 16, 18, 32], [20, 11, 5, 13, 19], [8, 31, 20, 14, 5], [8, 25, 31, 1, 6], [14, 12, 4, 31, 1], [25, 6, 12, 32, 21], [21, 7, 18, 29, 9]  …  [86, 98, 47, 90, 57], [84, 97, 79, 94, 93], [86, 101, 46, 57, 90], [78, 82, 93, 79, 99], [78, 79, 94, 82, 99], [88, 73, 72, 74, 85], [46, 86, 101, 53, 43], [79, 95, 84, 94, 99], [94, 84, 97, 92, 79], [95, 99, 79, 81, 94]], [[0.14142135623730964, 0.14142135623730986, 0.1414213562373099, 0.17320508075688784, 0.24494897427831822], [0.22360679774997871, 0.264575131106459, 0.3162277660168373, 0.31622776601683805, 0.33166247903553986], [0.14142135623730948, 0.29999999999999966, 0.31622776601683816, 0.34641016151377546, 0.36055512754639873], [0.22360679774997858, 0.22360679774997916, 0.2999999999999998, 0.2999999999999998, 0.30000000000000027], [0.3605551275463992, 0.5477225575051663, 0.6164414002968979, 0.6403124237432853, 0.6557438524302004], [0.3464101615137753, 0.3605551275463989, 0.3

In [65]:
c = ytrain[hcat(idxs...)]
possible_labels = map(i->counter(c[:,i]),1:size(c,2))
predictions_NN = map(i->parse(Int,string(string(argmax(possible_labels[i])))),1:size(c,2))
findaccuracy(predictions_NN,y[testids])

0.9591836734693877

### 🟣 Method 7: Support Vector Machines
We will use the `LIBSVM` package here.

In [66]:
Xtrain = X[trainids,:]
ytrain = y[trainids]

101-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3

In [67]:
model = svmtrain(Xtrain', ytrain)

LIBSVM.SVM{Int64, LIBSVM.Kernel.KERNEL}(SVC, LIBSVM.Kernel.RadialBasis, nothing, 4, 101, 3, [1, 2, 3], Int32[1, 2, 3], Float64[], Int32[], LIBSVM.SupportVectors{Vector{Int64}, Matrix{Float64}}(36, Int32[5, 15, 16], [1, 1, 1, 1, 1, 2, 2, 2, 2, 2  …  3, 3, 3, 3, 3, 3, 3, 3, 3, 3], [4.3 5.8 … 6.5 5.9; 3.0 4.0 … 3.0 3.0; 1.1 1.2 … 5.2 5.1; 0.1 0.2 … 2.0 1.8], Int32[10, 11, 13, 16, 28, 33, 35, 37, 38, 40  …  86, 87, 89, 90, 91, 95, 96, 98, 99, 101], LIBSVM.SVMNode[LIBSVM.SVMNode(1, 4.3), LIBSVM.SVMNode(1, 5.8), LIBSVM.SVMNode(1, 5.7), LIBSVM.SVMNode(1, 4.8), LIBSVM.SVMNode(1, 5.1), LIBSVM.SVMNode(1, 7.0), LIBSVM.SVMNode(1, 6.9), LIBSVM.SVMNode(1, 6.5), LIBSVM.SVMNode(1, 6.3), LIBSVM.SVMNode(1, 5.0)  …  LIBSVM.SVMNode(1, 6.2), LIBSVM.SVMNode(1, 7.2), LIBSVM.SVMNode(1, 7.9), LIBSVM.SVMNode(1, 6.3), LIBSVM.SVMNode(1, 6.1), LIBSVM.SVMNode(1, 6.9), LIBSVM.SVMNode(1, 5.8), LIBSVM.SVMNode(1, 6.3), LIBSVM.SVMNode(1, 6.5), LIBSVM.SVMNode(1, 5.9)]), 0.0, [0.5062933001989695 0.7301974967269038; 0.0372

In [68]:
predictions_SVM, decision_values = svmpredict(model, X[testids,:]')
findaccuracy(predictions_SVM,y[testids])

0.9591836734693877

Putting all the results together:

In [69]:
overall_accuracies = zeros(7)
methods = ["lasso","ridge","EN", "DT", "RF","kNN", "SVM"]
ytest = y[testids]
overall_accuracies[1] = findaccuracy(predictions_lasso,ytest)
overall_accuracies[2] = findaccuracy(predictions_ridge,ytest)
overall_accuracies[3] = findaccuracy(predictions_EN,ytest)
overall_accuracies[4] = findaccuracy(predictions_DT,ytest)
overall_accuracies[5] = findaccuracy(predictions_RF,ytest)
overall_accuracies[6] = findaccuracy(predictions_NN,ytest)
overall_accuracies[7] = findaccuracy(predictions_SVM,ytest)
hcat(methods, overall_accuracies)

7×2 Matrix{Any}:
 "lasso"  0.959184
 "ridge"  0.918367
 "EN"     0.959184
 "DT"     0.897959
 "RF"     0.918367
 "kNN"    0.959184
 "SVM"    0.959184

# Finally...
After finishing this notebook, you should be able to:
- [ ] split your data into training and testing data to test the effectiveness of a certain method
- [ ] apply a simple accuracy function to test the effectiveness of a certain method
- [ ] run multiple classification algorithms:
    - [ ] LASSO
    - [ ] Ridge
    - [ ] ElasticNet
    - [ ] Decision Tree
    - [ ] Random Forest
    - [ ] Nearest Neighbors
    - [ ] Support Vector Machines

# 🥳 One cool finding

We used multiple methods to run classification on the `iris` dataset which is a dataset of flowers and there are three types of iris flowers in it. We split the data into training and testing and ran our methods. Here is the scoreboard:

| method | accuracy score |
|---|---|
| lasso  |1.0|
| ridge  |1.0|
| EN     |1.0|
| DT     |0.960784|
| RF     |0.980392|
| kNN    |1.0|
| SVM    |1.0|